In [ ]:
!pip install transformers

In [ ]:
import io
import pandas as pd
import nltk
import torch
from transformers import BertTokenizer, BertForQuestionAnswering
from google.colab import files

In [ ]:
uploaded=files.upload()

Saving caseint_data.xlsx to caseint_data.xlsx


In [ ]:
data=pd.read_excel('caseint_data.xlsx')

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [1]:
def token_data(data):
    token_data=[]
    for question in data['Questions']:
        if pd.notnull(question):
            token_question=tokenizer.encode_plus(question, return_tensors='pt', max_length=512, truncation=True)
            token_data.append(token_question)
    return token_data

#questions from tokenized data
def gen_q(token_data):
    generated_questions= []
    for token_q in token_data:
        outputs = model(**token_q)
        start_scores=outputs.start_logits
        end_scores=outputs.end_logits

        start_index=torch.argmax(start_scores)
        end_index=torch.argmax(end_scores) + 1

        #remove special tokens
        start_index = max(start_index, 1)
        end_index = max(end_index, 1)
        start_index = min(start_index, len(token_q['input_ids'][0])-1)
        end_index = min(end_index, len(token_q['input_ids'][0])-1)

        generated_question = tokenizer.decode(token_q['input_ids'][0][start_index:end_index])
        generated_questions.append(generated_question)
    return generated_questions

tokenized_data = token_data(data)
generated_questions = gen_q(tokenized_data)

#extract keywords from a sentence
def extract_keywords(sentence):
    words = sentence.split()
    keywords = ' '.join(words[:3])
    return keywords

#top three entries
data1=data.head(3)

for question, answer in zip(generated_questions, data1['Questions']):
    if pd.notnull(answer):
        print("Question:", question)
        keywords = extract_keywords(question)
        print("Keywords:", keywords)
        print()


NameError: name 'data' is not defined

In [ ]:
def token_data(data):
    token_data=[]
    for question in data['Questions']:
        if pd.notnull(question):
            token_question=tokenizer.encode_plus(question, return_tensors='pt', max_length=512, truncation=True)
            token_data.append(token_question)
    return token_data

In [ ]:


#questions from tokenized data
def gen_q(token_data, data):
    generated_questions= []
    tokenized_answers=[]
    for token_q, answer in zip(token_data, data['Answers']):
        if pd.notnull(answer):
            outputs = model(**token_q)
            start_scores = outputs.start_logits
            end_scores = outputs.end_logits

            start_index = torch.argmax(start_scores)
            end_index = torch.argmax(end_scores) + 1

            # Remove special tokens
            start_index = max(start_index, 1)
            end_index = max(end_index, 1)
            start_index = min(start_index, len(token_q['input_ids'][0]) - 1)
            end_index = min(end_index, len(token_q['input_ids'][0]) - 1)

            generated_question = tokenizer.decode(token_q['input_ids'][0][start_index:end_index])
            generated_questions.append(generated_question)

            # Tokenize the answer
            tokenized_answer = tokenizer.encode_plus(answer, return_tensors='pt', max_length=512, truncation=True)
            tokenized_answers.append(tokenized_answer)

    return generated_questions, tokenized_answers

tokenized_data = token_data(data)
generated_questions = gen_q(tokenized_data,data)

#extract keywords from a sentence
def extract_keywords(token_ids):
    tokens = tokenizer.convert_ids_to_tokens(token_ids)
    # Select the first few tokens as keywords
    keywords = ' '.join(tokens[:3])  # Adjust the number of keywords as needed
    return keywords

data1=data.head(3)

for question, answer in zip(generated_questions, data1['answers']):
    print("Question:", question)
    keywords = extract_keywords(question)
    print("Keywords:", keywords)

    user_answer = input("Your answer: ").lower()
    user_answer_tokens = tokenizer.tokenize(user_answer)

    dataset_answer_tokens = tokenizer.tokenize(answer)

    #keyword match ratio
    keyword_match_ratio = len(set(user_answer_tokens) & set(dataset_answer_tokens)) / len(set(dataset_answer_tokens))
    print("Keyword match ratio:", keyword_match_ratio)

KeyError: 'Answers'

In [1]:
def token_data(data):
    token_data = []
    for question in data['Questions']:
        if pd.notnull(question):
            token_question = tokenizer.encode_plus(question, return_tensors='pt', max_length=512, truncation=True)
            token_data.append(token_question)
    return token_data

# Generate questions and tokenized answers from tokenized data
def gen_q(token_data, data):
    generated_questions = []
    tokenized_answers = []
    for token_q, answer in zip(token_data, data['answers']):
        if pd.notnull(answer):
            outputs = model(**token_q)
            start_scores = outputs.start_logits
            end_scores = outputs.end_logits

            start_index = torch.argmax(start_scores)
            end_index = torch.argmax(end_scores) + 1

            # Remove special tokens
            start_index = max(start_index, 1)
            end_index = max(end_index, 1)
            start_index = min(start_index, len(token_q['input_ids'][0]) - 1)
            end_index = min(end_index, len(token_q['input_ids'][0]) - 1)

            generated_question = tokenizer.decode(token_q['input_ids'][0][start_index:end_index])
            generated_questions.append(generated_question)

            # Tokenize the answer
            tokenized_answer = tokenizer.encode_plus(answer, return_tensors='pt', max_length=512, truncation=True)
            tokenized_answers.append(tokenized_answer)

    return generated_questions, tokenized_answers

# Tokenize the data
tokenized_data = token_data(data)

# Generate questions from tokenized data
generated_questions, tokenized_answers = gen_q(tokenized_data, data)

# Function to extract keywords from a list of token IDs
def extract_keywords(token_ids):
    tokens = tokenizer.convert_ids_to_tokens(token_ids)
    # Select the first few tokens as keywords
    keywords = ' '.join(tokens[:3])  # Adjust the number of keywords as needed
    return keywords

# Print questions and associated keywords
for question, tokenized_answer in zip(generated_questions, tokenized_answers):
    print("Question:", question)
    keywords = extract_keywords(tokenized_answer['input_ids'][0])
    print("Keywords:", keywords)
    print()

    user_answer = input("Your answer: ").lower()
    user_answer_tokens = tokenizer.tokenize(user_answer)

    dataset_answer_tokens = tokenizer.tokenize(answer)

    #keyword match ratio
    keyword_match_ratio = len(set(user_answer_tokens) & set(dataset_answer_tokens)) / len(set(dataset_answer_tokens))
    print("Keyword match ratio:", keyword_match_ratio)


NameError: name 'data' is not defined

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

topic = input("Enter the topic you want to interview about: ")

f_topic = topic.capitalize()
f_topic = '_'.join(f_topic.split()).lower()

url = f'https://en.wikipedia.org/wiki/{f_topic}'
response =requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

paragraphs = soup.find_all('p')
paragraph_texts = [p.get_text() for p in paragraphs]

df = pd.DataFrame({'Paragraphs': paragraph_texts})

def generate_questions(paragraph):
    inputs = tokenizer.encode_plus(paragraph, return_tensors="pt", add_special_tokens=True)
    start_logits, end_logits = model(**inputs).start_logits, model(**inputs).end_logits

    start_index = torch.argmax(start_logits, dim=1).item()
    end_index = torch.argmax(end_logits, dim=1).item()

    answer_tokens = inputs["input_ids"][0][start_index:end_index+1]
    answer = tokenizer.decode(answer_tokens)

    question = f"Analyse and answer: {answer}'?"

    return question
#Taking input (spoken response)
import speech_recognition as sr

class answers:
    def __init__(self):
        self.response = ""

    def record_response(self):
        recognizer = sr.Recognizer()
        with sr.Microphone() as source:
            print("Speak now:")
            recognizer.adjust_for_ambient_noise(source)  # Adjust for noise
            audio = recognizer.listen(source)

        try:
            self.response = recognizer.recognize_google(audio)
            print("You said:", self.response)
        except sr.UnknownValueError:
            print("Sorry, I couldn't understand what you said.")
        except sr.RequestError as e:
            print("Could not request results from Google Speech Recognition service; {0}".format(e))


# usage instance :
answers = answers()
answers.record_response()

# Now you can access the response using answers.response
print("Response stored:", answers.response)

df['Generated_Questions'] = df['Paragraphs'].apply(generate_questions)
def get_user_answers(questions):
    user_answers = []
    for question in questions:
        answer = input(f"Question: {question}\nYour Answer: ")
        user_answers.append(answer)
    return user_answers

#first 10 questions
questions = df['Generated_Questions'].head(10)
user_answers = get_user_answers(questions)

qa_df = pd.DataFrame({'Questions': questions, 'User_Answers': user_answers})

#DataFrame to an Excel file
qa_df.to_excel('user_answers.xlsx', index=False)


ModuleNotFoundError: No module named 'distutils'